In [ ]:
import openai
import os

from dotenv import find_dotenv, load_dotenv

from langsmith import traceable


_ = load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

MODEL_NAME = os.environ["MODEL_NAME"]
MODEL_PROVIDER = os.environ["MODEL_PROVIDER"]

In [ ]:
inputs = [
	{"role": "system", "content": "You are a helpful assistant."},
	{"role": "user", "content": "I'd like to book a table for two."},
]

output = {
	"choices": [
		{
			"message": {
				"role": "assistant",
				"content": "Sure, what time would you like to book the table for?"
			}
		}
	]
}

@traceable(
	run_type="llm", 
	metadata={
		"model_name": MODEL_NAME, 
		"model_provider": MODEL_PROVIDER
	}
)
def chat_model(messages: list):
	return output

chat_model(inputs)

{'choices': [{'message': {'role': 'assistant',
    'content': 'Sure, what time would you like to book the table for?'}}]}

In [ ]:
def _reduce_chunks(chunks: list):
    all_text = "".join([chunk["choices"][0]["message"]["content"] for chunk in chunks])
    return {"choices": [{"message": {"content": all_text, "role": "assistant"}}]}

@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER, 
        "ls_model_name": MODEL_NAME
    },
    reduce_fn=_reduce_chunks
)
def my_streaming_chat_model(messages: list):
    for chunk in ["Hello, " + messages[1]["content"]]:
        yield {
            "choices": [
                {
                    "message": {
                        "content": chunk,
                        "role": "assistant",
                    }
                }
            ]
        }

list(
    my_streaming_chat_model(
        [
            {"role": "system", "content": "You are a helpful assistant. Please greet the user."},
            {"role": "user", "content": "polly the parrot"},
        ],
    )
)

[{'choices': [{'message': {'content': 'Hello, polly the parrot',
     'role': 'assistant'}}]}]

In [5]:
def _convert_docs(results):
  return [
      {
          "page_content": r,
          "type": "Document",
          "metadata": {"foo": "bar"}
      }
      for r in results
  ]

@traceable(
    run_type="retriever"
)
def retrieve_docs(query):
  """Retriever returning hardcoded dummy documents. In production, this could be a real vector datatabase or other document index.
  """
  contents = ["Document contents 1", "Document contents 2", "Document contents 3"]
  return _convert_docs(contents)

retrieve_docs("User query.")

[{'page_content': 'Document contents 1',
  'type': 'Document',
  'metadata': {'foo': 'bar'}},
 {'page_content': 'Document contents 2',
  'type': 'Document',
  'metadata': {'foo': 'bar'}},
 {'page_content': 'Document contents 3',
  'type': 'Document',
  'metadata': {'foo': 'bar'}}]

In [6]:
import json

from openai import OpenAI
from typing import Dict, List, Optional


openai_client = OpenAI()

@traceable(
  run_type="tool"
)
def get_current_temperature(location: str, unit: str):
    return 65 if unit == "Fahrenheit" else 17


@traceable(run_type="llm")
def call_openai(
    messages: List[Dict], tools: Optional[List[Dict]]
) -> str:
	return openai_client.chat.completions.create(
		model=MODEL_NAME,
		messages=messages,
		temperature=0,
		tools=tools
	)


@traceable(run_type="chain")
def ask_about_the_weather(inputs, tools):
	response = call_openai(inputs, tools)
	tool_call_args = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
	location = tool_call_args["location"]
	unit = tool_call_args["unit"]
	tool_response_message = {
		"role": "tool",
		"content": json.dumps(
			{
				"location": location,
				"unit": unit,
				"temperature": get_current_temperature(location, unit),
			}
		),
		"tool_call_id": response.choices[0].message.tool_calls[0].id
	}
	inputs.append(response.choices[0].message)
	inputs.append(tool_response_message)
	output = call_openai(inputs, None)
	return output


tools = [
	{
		"type": "function",
		"function": {
			"name": "get_current_temperature",
			"description": "Get the current temperature for a specific location",
			"parameters": {
				"type": "object",
				"properties": {
					"location": {
						"type": "string",
						"description": "The city and state, e.g., San Francisco, CA"
					},
					"unit": {
						"type": "string",
						"enum": ["Celsius", "Fahrenheit"],
						"description": "The temperature unit to use. Infer this from the user's location."
					}
				},
				"required": ["location", "unit"]
			}
		}
	}
]
inputs = [
  {"role": "system", "content": "You are a helpful assistant."},
  {"role": "user", "content": "What is the weather today in New York City?"},
]

ask_about_the_weather(inputs, tools)

ChatCompletion(id='chatcmpl-BHlF3yPJHvLGdL28wRc0Nrmv99HQO', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current temperature in New York City is 65°F. If you need more detailed weather information, feel free to ask!', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[]))], created=1743572873, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_b376dfbbd5', usage=CompletionUsage(completion_tokens=26, prompt_tokens=83, total_tokens=109, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))